In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pakistan-intellectual-capital/pakistan_intellectual_capital.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import chardet
from scipy import stats
from mlxtend.preprocessing import minmax_scaling
import fuzzywuzzy
from fuzzywuzzy import process
%matplotlib Inline
%config InlineBacked.figure_format='retina'
pd.set_option('display.max_columns',100)
np.random.seed(0)
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('../input/pakistan-intellectual-capital/pakistan_intellectual_capital.csv')
df.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [4]:
univesity=df['University Currently Teaching'].unique()
univesity.sort()
univesity

array(['Abasyn University', 'Air University',
       'Bahria University,Islamabad',
       'Barani Institute of Information and Technology',
       'Beaconhouse National University', 'COMSATS, Islamabad Campus',
       'Capital University of Science and Technology', 'City University',
       'DHA Suffa University', 'FAST(Faisalabad)', 'FAST(Karachi)',
       'FAST(Lahore)', 'FAST,Peshawar', 'FAST-NU(Islamabad)',
       'Fatima Jinnah Women University', 'Ghulam Ishaq Khan Institute',
       'Gift University', 'Government College University Faisalabad',
       'Habib University', 'ISRA University',
       'Information Technology University',
       'Institute of Business Administration, Karachi\xa0',
       'Institute of Business Administration,Sukkur',
       'International Islamic University,Islamabad',
       'Islamia University Bhawalpur', 'Jinnah University for Women',
       'Lahore Garrison University',
       'Lahore University of Management Sciences',
       'Minhaj University L

In [5]:
df['University Currently Teaching']=df['University Currently Teaching'].str.lower()
df['University Currently Teaching']=df['University Currently Teaching'].str.strip()

In [6]:
unique_data=df['Country'].unique()
unique_data.sort()
unique_data

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

In [7]:
df['Country']=df['Country'].str.lower()
df['Country']=df['Country'].str.strip()

In [8]:
match=fuzzywuzzy.process.extract('South korea',unique_data,limit=10,scorer=fuzzywuzzy.fuzz.token_sort_ratio)
match

[('South Korea', 100),
 ('SouthKorea', 48),
 ('Saudi Arabia', 43),
 ('Norway', 35),
 ('Austria', 33),
 ('Ireland', 33),
 ('Pakistan', 32),
 ('Portugal', 32),
 ('Scotland', 32),
 ('Australia', 30)]

In [9]:
def replace_sting(df,columns,text,ratio=47):
    string=df[columns].unique()
    match=fuzzywuzzy.process.extract(text,string,limit=10,scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    string_match=[i[0] for i in match if i[1]>=ratio]
    row_match=df[columns].isin(string_match)
    df.loc[row_match,columns]=text
    print('Done')


In [10]:
replace_sting(df,'Country','South korea')

Done


In [11]:
df['Country'].unique()

array(['thailand', 'pakistan', 'germany', 'austria', 'australia', 'uk',
       'china', 'france', 'usofa', 'South korea', 'malaysia', 'sweden',
       'italy', 'canada', 'norway', 'ireland', 'new zealand', 'urbana',
       'portugal', 'russian federation', 'usa', 'finland', 'netherland',
       'greece', 'turkey', 'macau', 'singapore', 'spain', 'japan',
       'hongkong', 'saudi arabia', 'mauritius', 'scotland'], dtype=object)

In [12]:
string_df=df['Terminal Degree'].unique()

string_df

array(['PhD', 'BS', 'MCS', 'MS', 'BE', 'Phd', nan, 'PostDoc', 'Masters',
       'MBA', 'MSc', 'BIT', 'MA', 'Mphil', 'MPhil', 'Post Doc', ' MCS',
       'MIT', 'BSC', 'MSCS', 'BSIT', 'MSC', 'M.Phil', 'Master',
       'Bachelors', 'M.Phil leading to PhD', 'BSCS', 'M.Sc', 'BCS', 'ME',
       'B.Ed', 'M.A', 'M.Eng', 'MCIT', 'B.E', 'M.E', 'MSIT', 'Ph.D',
       'MSSE', 'MSTN', 'Doctor of Professional Studies', 'BSc', 'BA'],
      dtype=object)

In [13]:
df['Terminal Degree']=df['Terminal Degree'].str.lower()
df['Terminal Degree']=df['Terminal Degree'].str.strip()

In [14]:
df['Terminal Degree'].unique()

array(['phd', 'bs', 'mcs', 'ms', 'be', nan, 'postdoc', 'masters', 'mba',
       'msc', 'bit', 'ma', 'mphil', 'post doc', 'mit', 'bsc', 'mscs',
       'bsit', 'm.phil', 'master', 'bachelors', 'm.phil leading to phd',
       'bscs', 'm.sc', 'bcs', 'me', 'b.ed', 'm.a', 'm.eng', 'mcit', 'b.e',
       'm.e', 'msit', 'ph.d', 'msse', 'mstn',
       'doctor of professional studies', 'ba'], dtype=object)

In [15]:
match=fuzzywuzzy.process.extract('bs',string_df,limit=10,scorer=fuzzywuzzy.fuzz.token_sort_ratio)
match

[('BS', 100),
 ('BSC', 80),
 ('BCS', 80),
 ('BSc', 80),
 ('BSIT', 67),
 ('BSCS', 67),
 ('MS', 50),
 ('BE', 50),
 ('BA', 50),
 ('MCS', 40)]

In [16]:
def replace_string(text,df,columns,string):
    row_match=df[columns].isin(string)
    df.loc[row_match,columns]=text

In [17]:
match_string=['bs','bsc','bcs','bscs']
replace_string('BS',df,'Terminal Degree',match_string)

In [18]:
df['Terminal Degree'].unique()

array(['phd', 'BS', 'mcs', 'ms', 'be', nan, 'postdoc', 'masters', 'mba',
       'msc', 'bit', 'ma', 'mphil', 'post doc', 'mit', 'mscs', 'bsit',
       'm.phil', 'master', 'bachelors', 'm.phil leading to phd', 'm.sc',
       'me', 'b.ed', 'm.a', 'm.eng', 'mcit', 'b.e', 'm.e', 'msit', 'ph.d',
       'msse', 'mstn', 'doctor of professional studies', 'ba'],
      dtype=object)